Устанавливаем нужные библиотеки

In [1]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.5/485.5 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 10.3 MB/s eta 0:00:00


In [2]:
import os
import shutil
from ultralytics import YOLO

Подключаемся к своему Google Drive

In [3]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [38]:
# записываем необходимые пути
RootDir='/content/gdrive/MyDrive/ColabNotebooks/MenOnCamera'
train_path_img = "/content/yolo_data/images/train/"
train_path_label = "/content/yolo_data/labels/train/"
val_path_img = "/content/yolo_data/images/val/"
val_path_label = "/content/yolo_data/labels/val/"
test_path = "/content/yolo_data/test"
model_path=f"{RootDir}/runs/train/trainXL_118ep_2400ph/weights/last.pt"


Разделяем dataset на тренировочные и валидационные данные

In [5]:
def GoToPath(path):
    os.chdir(RootDir+path)
    files=os.listdir()
    files_cnt=len(files)
    print(files_cnt)

    return(files,files_cnt)

In [6]:
def train_test_split(RootDir, split=0.2):
    print("------ PROCESS STARTED -------")

    # создаем нужные директории

    os.makedirs(train_path_img, exist_ok=True)
    os.makedirs(train_path_label, exist_ok=True)
    os.makedirs(val_path_img, exist_ok=True)
    os.makedirs(val_path_label, exist_ok=True)


    # копируем тренировочные фотографии
    files,files_cnt=GoToPath("/data/images/train/")

    num = 0
    for img in files:
        if img[-4:]==".jpg":
          img2=img.replace(".jpg",".txt")
          if num >= files_cnt * (1 - split):
                shutil.copy(RootDir + "/data/images/train/" + img, val_path_img + img)
                shutil.copy(RootDir + "/data/labels/train/" + img2, val_path_label + img2)
          else:
              shutil.copy(RootDir + "/data/images/train/" + img, train_path_img + img)
              shutil.copy(RootDir + "/data/labels/train/" + img2, train_path_label + img2)

          num += 1

    print("------ TASK COMPLETED -------")

In [ ]:
# удаление копии папки с dataset
shutil.rmtree("/content/yolo_data")

In [34]:
train_test_split(RootDir)

------ PROCESS STARTED -------
2785
------ TASK COMPLETED -------


Тренируем нейросеть

In [35]:
model = YOLO("yolov8x.yaml")
# model = YOLO(model_path)

results=model.train(data=os.path.join(RootDir,"config.yaml"),epochs=300)


                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.Conv                  [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.Conv                  [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.C2f                   [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.Conv                  [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.C2f                   [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.Conv                  [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.C2f                   [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics.nn.modules.Conv                  [640

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.Conv                  [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.Conv                  [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.C2f                   [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.Conv                  [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.C2f                   [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.Conv                  [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.C2f                   [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics

  0%|          | 0.00/6.23M [00:00<?, ?B/s]

AMP: checks passed ✅
optimizer: SGD(lr=0.01) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias
train: Scanning /content/yolo_data/labels/train... 2228 images, 67 backgrounds, 0 corrupt: 100%|██████████| 2243/2243 [00:01<00:00, 1583.87it/s]
train: New cache created: /content/yolo_data/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Scanning /content/yolo_data/labels/val... 557 images, 18 backgrounds, 0 corrupt: 100%|██████████| 557/557 [00:00<00:00, 1586.32it/s]
val: New cache created: /content/yolo_data/labels/val.cache
Plotting labels to runs/detect/train/labels.jpg... 
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/detect/train
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      1/300        13G      5.539      6.947 

In [39]:
model = YOLO(model_path)

metrics = model.val()  
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Ultralytics YOLOv8.0.55 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x summary (fused): 268 layers, 68124531 parameters, 0 gradients, 257.4 GFLOPs
val: Scanning /content/yolo_data/labels/val.cache... 557 images, 18 backgrounds, 0 corrupt: 100%|██████████| 557/557 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:32<00:00,  1.06it/s]
                   all        557        581      0.991      0.993      0.994      0.587
Speed: 0.3ms preprocess, 50.7ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/val


array([    0.58713])